In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action="ignore")

### Kernel specs
* leveraging Python 3 (Data Science) out of the box

In [25]:
%%capture
!pip3 install hydra-core==1.1.1

In [7]:
!pip3 show sagemaker

Name: sagemaker
Version: 2.70.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, numpy, packaging, pandas, pathos, protobuf, protobuf3-to-dict, smdebug-rulesconfig
Required-by: 


In [38]:
import sagemaker, boto3, os, time, tarfile
from sagemaker.huggingface import HuggingFace
import pandas as pd
from IPython.display import display
from omegaconf import OmegaConf

parent_dir = os.path.dirname(os.getcwd())
conf = OmegaConf.load(os.path.join(parent_dir,'conf/config.yaml'))

In [36]:
sagemaker_session = sagemaker.Session()
s3_bucket = sagemaker_session.default_bucket()
data_prefix = conf['ag-news']['data_prefix']
iam_role = sagemaker.get_execution_role()
aws_region = boto3.Session().region_name

### [AWS Open Data Registry](https://registry.opendata.aws/fast-ai-nlp/)

* AG news dataset for text classification 

In [13]:
!wget -nc https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
tf = tarfile.open('ag_news_csv.tgz')
tf.extractall()
!rm -fr ag_news_csv.tgz

--2022-04-03 05:43:46--  https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.79.142
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.79.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11784419 (11M) [application/x-tar]
Saving to: ‘ag_news_csv.tgz’

ag_news_csv.tgz     100%[===================>]  11.24M  28.3MB/s    in 0.4s    

2022-04-03 05:43:47 (28.3 MB/s) - ‘ag_news_csv.tgz’ saved [11784419/11784419]



In [40]:
std_cols = conf['ag-news']['std_cols']
data_folder = conf['ag-news']['data_folder']
train = pd.read_csv(f'{parent_dir}/notebooks/{data_folder}/train.csv',names=std_cols)
test = pd.read_csv(f'{parent_dir}/notebooks/{data_folder}/test.csv',names=std_cols)
train.to_csv(f'{parent_dir}/notebooks/{data_folder}/ag-train.csv',index=False)
test.to_csv(f'{parent_dir}/notebooks/{data_folder}/ag-test.csv',index=False)